In [21]:
import numpy as np

import torch

import chess
import chess.svg
import chess.engine

from ts_alpha_zero import AlphaChess, AlphaChessHelper, MCTS, AlphaZero, ResNet, display_board
        

In [22]:
import random


alphaChess = AlphaChess()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


args = {
    'num_searches': 100,
    'num_iterations': 1,
    'num_selfPlay_iterations': 2000,
    'num_epochs': 300,
    'batch_size': 256,
    'temperature': 1.25,
    'dirichlet_epsilon': 0.25,
    'dirichlet_alpha': 0.3
}
# learn_fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
# learn_fen = "RR6/8/3K4/8/8/4k3/8/6rr w - - 0 1"
learn_fen = "8/8/8/3K4/2B5/6p1/7k/5R2 w - - 0 1"
# learn_fen = "8/8/8/3K4/8/5R2/8/5Bnk w - - 0 1"
# learn_fen = "RR6/8/8/3K4/8/5k2/8/8 w - - 0 1"
stockfish_engine = chess.engine.SimpleEngine.popen_uci("./stockfish/stockfish-ubuntu-x86-64")
leela_engine = chess.engine.SimpleEngine.popen_uci(
    "lc0/lc0")
leela_engine.configure({"WeightsFile": "lc0/BT4-1740.pb"})

save_plays = False
learn = False
load = True
test = True 
model = ResNet(AlphaChessHelper.ACTION_SIZE, 9, 128, device=device)
if load:
    model.load_state_dict(torch.load('model_2_AlphaChess.pt', map_location=device))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
AlphaChessHelper.stockfish_engine = stockfish_engine
AlphaChessHelper.leela_engine = leela_engine

alphaZero = AlphaZero(model, optimizer, alphaChess, args)
    
if save_plays:
    alphaZero.play_and_save([learn_fen], ["iter_7"])

if learn:
    alphaZero.learn(play_iters=["iter_7"], iteration=2)

if test:
    alphaChess.reset(learn_fen)

    mcts = MCTS(alphaChess, args, model)

    state = alphaChess.get_initial_state()
    player = 1

    while True:
        
        if player == -1:
            if False:
                valid_moves = AlphaChessHelper.get_valid_moves(alphaChess.board)
                # print first 10 valid moves
                print("valid_moves", [AlphaChessHelper.integer_to_move(move) for move in valid_moves][:10])
                action = chess.Move.from_uci(str(input(f"{player}:")))

                if AlphaChessHelper.move_to_integer(action) not in valid_moves:
                    print("action not valid")
                    continue
            else:
                action = AlphaChessHelper.engine_play(alphaChess.board)
            
        else:
            print("AI is thinking...")
            neutral_state = AlphaChessHelper.get_encoded_state(alphaChess.board)
            mcts_probs = mcts.search(neutral_state, 1)
            action = AlphaChessHelper.integer_to_move(np.argmax(mcts_probs))
            print("AI moved", action)
        alphaChess.move(action)
        value, is_terminal = AlphaChessHelper.get_value_and_terminated(alphaChess.board)
        display_board(alphaChess.board)
        if is_terminal:
            if value == 1:
                print(player, "won")
            elif value == -1:
                print(player, "lost")
            else:
                print("draw")
            break
        player *= -1
            


<UciProtocol (pid=73298)>: stderr >>        _
<UciProtocol (pid=73298)>: stderr >> |   _ | |
<UciProtocol (pid=73298)>: stderr >> |_ |_ |_| v0.32.0-dev+git.2634784 built Oct  2 2024


OutOfMemoryError: CUDA out of memory. Tried to allocate 350.00 MiB (GPU 0; 8.00 GiB total capacity; 6.47 GiB already allocated; 0 bytes free; 7.14 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF